# Module 3: From LangGraph Nodes to LangChain Tools with @tool Decorator

**Building on Modules 1 & 2**:
- Module 1: Built agents using `create_agent()` with plain Python functions
- Module 2: Learned LangGraph fundamentals with nodes and edges
- Module 3: **Transform nodes into professional tools using @tool decorator**

**Why use @tool decorator?**
- Automatic schema generation for LLM tool calling
- Better documentation and type validation
- Seamless integration with LangChain ecosystem
- Enhanced error handling and debugging
- Industry-standard approach for LangChain 1.0

**Time:** 2-3 hours

## Setup: Install Dependencies

In [1]:
# Install LangChain 1.0 alpha packages
!pip install --pre -U langchain langchain-openai langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.8/467.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.78
    Uninstalling langchain-core-0.3.78:
      Successfully uninstalled langchain-core-0.3.78
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27


## Setup: Configure OpenAI API Key

In [2]:
# Retrieve the API key from Colab's secrets
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

print("✅ API Key configured!")

✅ API Key configured!


---
# Lab 1: Understanding the Difference - Plain Functions vs @tool

**Objective:** Compare plain Python functions (used as nodes) with @tool decorated functions.

**Scenario:** Same HR tools from Module 1, but showing the evolution

## Part 1: Plain Functions (The Old Way)

In [ ]:
# Old approach: Plain Python functions
def get_employee_info(employee_id: str) -> str:
    """Get employee information by ID."""
    employees = {
        "101": "Priya Sharma - Engineering - Senior Developer",
        "102": "Rahul Verma - Engineering - Manager",
        "103": "Anjali Patel - HR - Director",
        "104": "Arjun Reddy - Sales - Team Lead",
        "105": "Sneha Gupta - Marketing - Specialist"
    }
    return employees.get(employee_id, f"Employee {employee_id} not found")

def check_leave_balance(employee_id: str) -> str:
    """Check remaining leave days for an employee by ID."""
    leave_data = {
        "101": "Priya Sharma has 12 days of leave remaining",
        "102": "Rahul Verma has 8 days of leave remaining",
        "103": "Anjali Patel has 15 days of leave remaining",
        "104": "Arjun Reddy has 10 days of leave remaining",
        "105": "Sneha Gupta has 5 days of leave remaining"
    }
    return leave_data.get(employee_id, f"Leave data for employee {employee_id} not found")

# Test plain functions
print("Plain Function Results:")
print("=" * 50)
print(get_employee_info("101"))
print(check_leave_balance("101"))
print("\n⚠️  Problem: No automatic schema, no validation, no integration with LLM tool calling")

## Part 2: @tool Decorator (The New Way) ✨

**Key Benefits:**
- Automatic schema generation from type hints
- Built-in documentation from docstrings
- Ready for LLM tool calling
- Better error handling

In [3]:
from langchain_core.tools import tool
from typing import Annotated

# New approach: @tool decorator
@tool
def get_employee_info_tool(employee_id: Annotated[str, "The unique employee ID to look up"]) -> str:
    """Get employee information by ID. Returns name, department, and position."""
    employees = {
        "101": "Priya Sharma - Engineering - Senior Developer",
        "102": "Rahul Verma - Engineering - Manager",
        "103": "Anjali Patel - HR - Director",
        "104": "Arjun Reddy - Sales - Team Lead",
        "105": "Sneha Gupta - Marketing - Specialist"
    }
    return employees.get(employee_id, f"Employee {employee_id} not found")

@tool
def check_leave_balance_tool(employee_id: Annotated[str, "The employee ID to check leave balance for"]) -> str:
    """Check remaining leave days for an employee. Returns the number of leave days available."""
    leave_data = {
        "101": "Priya Sharma has 12 days of leave remaining",
        "102": "Rahul Verma has 8 days of leave remaining",
        "103": "Anjali Patel has 15 days of leave remaining",
        "104": "Arjun Reddy has 10 days of leave remaining",
        "105": "Sneha Gupta has 5 days of leave remaining"
    }
    return leave_data.get(employee_id, f"Leave data for employee {employee_id} not found")

# Inspect the tool properties
print("@tool Decorated Function Properties:")
print("=" * 50)
print(f"Tool Name: {get_employee_info_tool.name}")
print(f"Description: {get_employee_info_tool.description}")
print(f"Args Schema: {get_employee_info_tool.args}")
print("\n✅ Benefits: Auto schema, validation, LLM-ready!")

@tool Decorated Function Properties:
Tool Name: get_employee_info_tool
Description: Get employee information by ID. Returns name, department, and position.
Args Schema: {'employee_id': {'description': 'The unique employee ID to look up', 'title': 'Employee Id', 'type': 'string'}}

✅ Benefits: Auto schema, validation, LLM-ready!


## Part 3: Using Tools - Same Interface!

In [4]:
# Tools can be called just like regular functions
print("Direct Tool Invocation:")
print("=" * 50)
result1 = get_employee_info_tool.invoke({"employee_id": "101"})
print(result1)

result2 = check_leave_balance_tool.invoke({"employee_id": "101"})
print(result2)

print("\n✅ Same functionality, but now with schema and validation!")

Direct Tool Invocation:
Priya Sharma - Engineering - Senior Developer
Priya Sharma has 12 days of leave remaining

✅ Same functionality, but now with schema and validation!


---
# Lab 2: Using @tool with LangChain Agents

**Objective:** Integrate @tool decorated functions with LangChain agents.

**This is the power of @tool!** LLMs can now understand and use these tools automatically.

In [6]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# Create agent with @tool decorated functions
tools = [get_employee_info_tool, check_leave_balance_tool]

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=tools,
    system_prompt="You are an HR assistant. Help users with employee information and leave balance queries."
)

print("✅ Agent created with @tool decorated functions!")
print(f"Number of tools available: {len(tools)}")

✅ Agent created with @tool decorated functions!
Number of tools available: 2


## Test the Agent

In [7]:
# Test 1: Simple query
print("Test 1: Who is employee 101?")
print("=" * 70)
result = agent.invoke({
    "messages": [{"role": "user", "content": "Who is employee 101?"}]
})
print(result['messages'][-1].content)

print("\n" + "=" * 70)

# Test 2: Complex query requiring multiple tools
print("\nTest 2: Tell me about employee 102 and their leave balance")
print("=" * 70)
result = agent.invoke({
    "messages": [{"role": "user", "content": "Tell me about employee 102 and their leave balance"}]
})
print(result['messages'][-1].content)

print("\n✅ Agent automatically selected the right tools!")

Test 1: Who is employee 101?
Employee 101 is Priya Sharma. She works in the Engineering department as a Senior Developer.


Test 2: Tell me about employee 102 and their leave balance
Employee 102 is Rahul Verma, who works in the Engineering department as a Manager. He has 8 days of leave remaining.

✅ Agent automatically selected the right tools!


---
# Lab 3: Advanced @tool Features

**Objective:** Explore advanced @tool decorator features:
- Custom tool names
- Custom descriptions
- Pydantic models for complex inputs
- Error handling

## Part 1: Custom Names and Descriptions

In [8]:
from pydantic import BaseModel, Field

# Custom tool with explicit name
@tool("search_employee")
def find_employee(query: Annotated[str, "Employee name or ID to search for"]) -> str:
    """Search for an employee by name or ID. Returns detailed employee information."""
    employees = {
        "priya": "Employee ID: 101 - Priya Sharma - Engineering - Senior Developer",
        "rahul": "Employee ID: 102 - Rahul Verma - Engineering - Manager",
        "101": "Employee ID: 101 - Priya Sharma - Engineering - Senior Developer",
        "102": "Employee ID: 102 - Rahul Verma - Engineering - Manager"
    }
    query_lower = query.lower()
    return employees.get(query_lower, f"No employee found matching '{query}'")

print(f"Tool Name: {find_employee.name}")
print(f"Description: {find_employee.description}")
print("\n✅ Custom name makes it clearer for LLMs to choose the right tool!")

Tool Name: search_employee
Description: Search for an employee by name or ID. Returns detailed employee information.

✅ Custom name makes it clearer for LLMs to choose the right tool!


## Part 2: Using Pydantic Models for Complex Inputs

In [9]:
class LeaveRequestInput(BaseModel):
    """Input schema for submitting a leave request."""
    employee_id: str = Field(description="The employee ID submitting the leave request")
    start_date: str = Field(description="Leave start date in YYYY-MM-DD format")
    end_date: str = Field(description="Leave end date in YYYY-MM-DD format")
    reason: str = Field(description="Reason for the leave request")

@tool(args_schema=LeaveRequestInput)
def submit_leave_request(employee_id: str, start_date: str, end_date: str, reason: str) -> str:
    """Submit a leave request for an employee. Returns confirmation with request ID."""
    import random
    request_id = f"LR-{random.randint(1000, 9999)}"
    return f"Leave request {request_id} submitted successfully for employee {employee_id} from {start_date} to {end_date}. Reason: {reason}"

# Inspect the schema
print("Tool with Pydantic Schema:")
print("=" * 50)
print(f"Tool Name: {submit_leave_request.name}")
print(f"Args Schema: {submit_leave_request.args}")
print("\n✅ Pydantic provides rich validation and documentation!")

Tool with Pydantic Schema:
Tool Name: submit_leave_request
Args Schema: {'employee_id': {'description': 'The employee ID submitting the leave request', 'title': 'Employee Id', 'type': 'string'}, 'start_date': {'description': 'Leave start date in YYYY-MM-DD format', 'title': 'Start Date', 'type': 'string'}, 'end_date': {'description': 'Leave end date in YYYY-MM-DD format', 'title': 'End Date', 'type': 'string'}, 'reason': {'description': 'Reason for the leave request', 'title': 'Reason', 'type': 'string'}}

✅ Pydantic provides rich validation and documentation!


## Test the Complex Tool

In [10]:
# Test the tool
result = submit_leave_request.invoke({
    "employee_id": "101",
    "start_date": "2025-11-01",
    "end_date": "2025-11-05",
    "reason": "Vacation"
})

print("Leave Request Result:")
print("=" * 50)
print(result)
print("\n✅ Complex inputs handled with ease!")

Leave Request Result:
Leave request LR-5913 submitted successfully for employee 101 from 2025-11-01 to 2025-11-05. Reason: Vacation

✅ Complex inputs handled with ease!


## Part 3: Error Handling in Tools

In [12]:
from langchain_core.tools import tool, ToolException

@tool
def calculate_salary(employee_id: Annotated[str, "Employee ID to calculate salary for"]) -> str:
    """Calculate monthly salary for an employee. Returns salary details."""
    salaries = {
        "101": "Monthly Salary: ₹1,00,000",
        "102": "Monthly Salary: ₹1,50,000",
        "103": "Monthly Salary: ₹2,00,000"
    }

    if employee_id not in salaries:
        raise ToolException(f"Salary information not available for employee {employee_id}")

    return salaries[employee_id]

# Test with valid ID
print("Test with valid employee ID:")
try:
    result = calculate_salary.invoke({"employee_id": "101"})
    print(f"✅ {result}")
except Exception as e:
    print(f"❌ Error: {e}")

# Test with invalid ID
print("\nTest with invalid employee ID:")
try:
    result = calculate_salary.invoke({"employee_id": "999"})
    print(f"✅ {result}")
except ToolException as e:
    print(f"❌ Tool Exception: {e}")

print("\n✅ Proper error handling makes tools more robust!")

Test with valid employee ID:
✅ Monthly Salary: ₹1,00,000

Test with invalid employee ID:
❌ Tool Exception: Salary information not available for employee 999

✅ Proper error handling makes tools more robust!


---
# Lab 4: Integrating @tool with LangGraph Workflows

**Objective:** Combine LangGraph workflows with @tool decorated functions.

**This is the BEST of both worlds:**
- LangGraph for workflow control
- @tool for LLM-ready functions

In [13]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Step 1: Define Tools

In [14]:
# Define HR tools with @tool decorator
@tool
def validate_employee_id(employee_id: Annotated[str, "Employee ID to validate"]) -> str:
    """Validate if an employee ID exists in the system."""
    valid_ids = ["101", "102", "103", "104", "105"]
    if employee_id in valid_ids:
        return f"✅ Employee ID {employee_id} is valid"
    return f"❌ Employee ID {employee_id} is invalid"

@tool
def get_employee_details(employee_id: Annotated[str, "Employee ID to get details for"]) -> str:
    """Get detailed information about an employee."""
    details = {
        "101": "Priya Sharma | Engineering | Senior Developer | Joined: 2020",
        "102": "Rahul Verma | Engineering | Manager | Joined: 2018",
        "103": "Anjali Patel | HR | Director | Joined: 2015"
    }
    return details.get(employee_id, "Employee not found")

@tool
def calculate_benefits(employee_id: Annotated[str, "Employee ID to calculate benefits for"]) -> str:
    """Calculate employee benefits based on tenure and position."""
    return f"Benefits for {employee_id}: Health Insurance, PF, Bonus, Leave Encashment"

print("✅ All tools defined with @tool decorator!")
print(f"Tools: {validate_employee_id.name}, {get_employee_details.name}, {calculate_benefits.name}")

✅ All tools defined with @tool decorator!
Tools: validate_employee_id, get_employee_details, calculate_benefits


## Step 2: Define Workflow State

In [15]:
class EmployeeQueryState(TypedDict):
    """State for employee query workflow."""
    employee_id: str
    validation_result: str
    employee_details: str
    benefits: str
    messages: Annotated[list, add_messages]
    final_report: str

print("✅ Workflow state defined!")

✅ Workflow state defined!


## Step 3: Create Workflow Nodes Using Tools

In [16]:
def validate_node(state: EmployeeQueryState):
    """Node 1: Validate employee using @tool."""
    print(f"📋 Validating employee {state['employee_id']}...")
    result = validate_employee_id.invoke({"employee_id": state['employee_id']})
    return {
        "validation_result": result,
        "messages": [("assistant", result)]
    }

def fetch_details_node(state: EmployeeQueryState):
    """Node 2: Fetch employee details using @tool."""
    print(f"📄 Fetching details for employee {state['employee_id']}...")
    if "invalid" in state['validation_result'].lower():
        return {"employee_details": "Skipped - Invalid ID"}

    result = get_employee_details.invoke({"employee_id": state['employee_id']})
    return {
        "employee_details": result,
        "messages": [("assistant", result)]
    }

def calculate_benefits_node(state: EmployeeQueryState):
    """Node 3: Calculate benefits using @tool."""
    print(f"💰 Calculating benefits for employee {state['employee_id']}...")
    if "invalid" in state['validation_result'].lower():
        return {"benefits": "Skipped - Invalid ID"}

    result = calculate_benefits.invoke({"employee_id": state['employee_id']})
    return {
        "benefits": result,
        "messages": [("assistant", result)]
    }

def generate_report_node(state: EmployeeQueryState):
    """Node 4: Generate final report."""
    print(f"📊 Generating final report...")
    report = f"""
Employee Query Report
{'=' * 50}
Employee ID: {state['employee_id']}
Validation: {state['validation_result']}
Details: {state['employee_details']}
Benefits: {state['benefits']}
{'=' * 50}
    """
    return {
        "final_report": report,
        "messages": [("assistant", "Report generated successfully")]
    }

print("✅ All workflow nodes created using @tool functions!")

✅ All workflow nodes created using @tool functions!


## Step 4: Build the Workflow

In [17]:
# Create workflow
workflow = StateGraph(EmployeeQueryState)

# Add nodes
workflow.add_node("validate", validate_node)
workflow.add_node("fetch_details", fetch_details_node)
workflow.add_node("calculate_benefits", calculate_benefits_node)
workflow.add_node("generate_report", generate_report_node)

# Add edges
workflow.add_edge(START, "validate")
workflow.add_edge("validate", "fetch_details")
workflow.add_edge("fetch_details", "calculate_benefits")
workflow.add_edge("calculate_benefits", "generate_report")
workflow.add_edge("generate_report", END)

# Compile
app = workflow.compile()

print("✅ Workflow compiled!")
print("Flow: START → validate → fetch_details → calculate_benefits → generate_report → END")

✅ Workflow compiled!
Flow: START → validate → fetch_details → calculate_benefits → generate_report → END


## Step 5: Test the Workflow

In [18]:
# Test with valid employee
initial_state = {
    "employee_id": "101",
    "validation_result": "",
    "employee_details": "",
    "benefits": "",
    "messages": [],
    "final_report": ""
}

print("Running workflow for employee 101...")
print("=" * 70)
result = app.invoke(initial_state)
print("\nFinal Report:")
print(result['final_report'])

print("\n✅ Workflow completed using @tool decorated functions!")

Running workflow for employee 101...
📋 Validating employee 101...
📄 Fetching details for employee 101...
💰 Calculating benefits for employee 101...
📊 Generating final report...

Final Report:

Employee Query Report
Employee ID: 101
Validation: ✅ Employee ID 101 is valid
Details: Priya Sharma | Engineering | Senior Developer | Joined: 2020
Benefits: Benefits for 101: Health Insurance, PF, Bonus, Leave Encashment
    

✅ Workflow completed using @tool decorated functions!


---
# Summary: Plain Functions vs @tool Decorator

## Comparison Table

| Feature | Plain Functions | @tool Decorator |
|---------|----------------|------------------|
| **Schema Generation** | ❌ Manual | ✅ Automatic |
| **LLM Integration** | ❌ Limited | ✅ Native |
| **Type Validation** | ❌ Optional | ✅ Built-in |
| **Documentation** | ❌ Separate | ✅ Integrated |
| **Error Handling** | ❌ Manual | ✅ Enhanced |
| **Agent Compatibility** | ⚠️  Requires wrapper | ✅ Direct |
| **Best For** | Simple nodes | LLM-ready tools |

## When to Use @tool?

✅ **Use @tool when:**
- Building agents that need LLM tool calling
- Want automatic schema generation
- Need robust type validation
- Creating reusable tools
- Working with LangChain ecosystem

⚠️ **Use plain functions when:**
- Building simple workflow nodes
- Don't need LLM integration
- Internal processing only
- Maximum flexibility needed

---
# 🎯 Exercises

## Exercise 1: Create Custom Tools

Create three new @tool decorated functions:
1. `update_employee_info` - Update employee details
2. `approve_leave_request` - Approve/reject leave requests
3. `generate_payslip` - Generate monthly payslip

**Requirements:**
- Use Pydantic models for complex inputs
- Add proper docstrings
- Include error handling

In [ ]:
# Your code here
# Hint: Use @tool decorator and Pydantic BaseModel

@tool
def update_employee_info(employee_id: str, field: str, value: str) -> str:
    """TODO: Implement this tool."""
    pass

# TODO: Implement approve_leave_request and generate_payslip

## Exercise 2: Build a Tool-Powered Workflow

Create a LangGraph workflow that:
1. Takes an employee ID
2. Validates the employee
3. Checks their leave balance
4. Processes a leave request if balance is sufficient
5. Generates a confirmation report

**Use only @tool decorated functions!**

In [ ]:
# Your code here
# Hint: Define tools first, then create workflow nodes

# TODO: Define tools
# TODO: Create State
# TODO: Create nodes
# TODO: Build workflow
# TODO: Test

## Exercise 3: Error Handling Challenge

Enhance the `calculate_salary` tool to:
1. Validate employee ID format (must be 3 digits)
2. Check if employee exists
3. Handle division errors when calculating bonuses
4. Return meaningful error messages

Test with: valid ID, invalid format, non-existent ID

In [ ]:
# Your code here
from langchain.tools.base import ToolException

@tool
def enhanced_calculate_salary(employee_id: str) -> str:
    """TODO: Add robust error handling."""
    # TODO: Validate ID format
    # TODO: Check if employee exists
    # TODO: Calculate salary with error handling
    pass

# TODO: Test cases

## 🌟 Bonus Challenge: Advanced Tool Integration

Create an agent that can:
1. Handle conversational queries about employees
2. Use multiple tools in sequence
3. Maintain context across queries
4. Generate comprehensive reports

Example queries to support:
- "Who is employee 101 and how many leave days do they have?"
- "Compare the leave balances of employees in Engineering"
- "Generate a benefits report for all managers"

In [ ]:
# Your code here
# This is open-ended - be creative!
# Hint: Combine create_agent, @tool, and LangGraph

---
# Conclusion

**What you learned:**
1. ✅ The difference between plain functions and @tool decorator
2. ✅ How to create tools with automatic schema generation
3. ✅ Using Pydantic models for complex inputs
4. ✅ Error handling in tools
5. ✅ Integrating @tool with LangGraph workflows
6. ✅ Building production-ready agents with tools

**Key Takeaways:**
- **@tool decorator** is the standard way to create LLM-ready tools
- **Automatic schema generation** saves time and reduces errors
- **Type hints and docstrings** are critical for tool quality
- **Tools work seamlessly** with both agents and workflows

**Next Steps:**
- Explore tool artifacts (Module 4)
- Learn about tool streaming (Module 5)
- Build multi-agent systems (Module 6)
- Deploy production agents (Module 7)

---
**Created with:** LangChain 1.0 + OpenAI + LangGraph

**References:**
- [LangChain Tools Documentation](https://python.langchain.com/docs/concepts/tools/)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)